<a href="https://colab.research.google.com/github/bhagyashreedhebe/COMP8240_MajorProject/blob/main/NewData_Pegasus_wikihow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install torch==1.8.2+cu111 torchvision==0.9.2+cu111 torchaudio===0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html

Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html
     |█████████████▌                  | 834.1 MB 1.4 MB/s eta 0:13:14tcmalloc: large alloc 1147494400 bytes == 0x563502d78000 @  0x7f7a1d576615 0x5634ff9484cc 0x5634ffa2847a 0x5634ff94b2ed 0x5634ffa3ce1d 0x5634ff9bee99 0x5634ff9b99ee 0x5634ff94cbda 0x5634ff9bed00 0x5634ff9b99ee 0x5634ff94cbda 0x5634ff9bb737 0x5634ffa3dc66 0x5634ff9badaf 0x5634ffa3dc66 0x5634ff9badaf 0x5634ffa3dc66 0x5634ff9badaf 0x5634ff94d039 0x5634ff990409 0x5634ff94bc52 0x5634ff9bec25 0x5634ff9b99ee 0x5634ff94cbda 0x5634ff9bb737 0x5634ff9b99ee 0x5634ff94cbda 0x5634ff9ba915 0x5634ff94cafa 0x5634ff9bac0d 0x5634ff9b99ee
     |█████████████████               | 1055.7 MB 1.3 MB/s eta 0:12:18tcmalloc: large alloc 1434370048 bytes == 0x5635473ce000 @  0x7f7a1d576615 0x5634ff9484cc 0x5634ffa2847a 0x5634ff94b2ed 0x5634ffa3ce1d 0x5634ff9bee99 0x5634ff9b99ee 0x5634ff94cbda 0x5634ff9bed00 0x5634ff9b99ee 0x5634ff94cbda 0x5634ff9bb737 0x5634ffa3dc66 0x5634f

In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.4 MB/s 


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.5 MB/s 
     |████████████████████████████████| 596 kB 69.3 MB/s 
     |████████████████████████████████| 56 kB 3.0 MB/s 
     |████████████████████████████████| 3.3 MB 43.9 MB/s 
     |████████████████████████████████| 895 kB 65.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install rouge-score

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch
from rouge_score import rouge_scorer
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re

In [ ]:
model_name = 'google/pegasus-wikihow'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [ ]:
def summarize_wikihow_text(source, source_link):
        input_wikihow = []
        predicted = []
        req = Request(source_link, headers={'User-Agent': 'XYZ/3.0'})
        source_url = urlopen(req, timeout=10).read()
        # print(source_url)
        soup = BeautifulSoup(source_url,'lxml')
        text = ''
        for paragraph in soup.find_all('p'):
            text += paragraph.text
        text = re.sub(r'\[.*?\]+', '', text)
        text = text.replace('\n', ' ')
        text = ' '.join(text.split())
        input_wikihow.append(text)
        input = ' '.join(map(str, input_wikihow))
        batch = tokenizer(input, truncation=True, padding='longest', return_tensors="pt").to(device)
        translated = model.generate(**batch)
        tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
        predicted = ' '.join(map(str, tgt_text)) 
        title = ""
        for paragraph in soup.find_all('h1'):
          title += paragraph.get_text()
          process_title = re.sub(r'\[.*?\]+', '', title)
          process_title = process_title.replace('\n', ' ')
          process_title = ' '.join(process_title.split())
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
        scores = scorer.score(str(process_title),str(predicted))
        recall = scores['rouge1'].recall
        precision = scores['rouge1'].precision
        fmeasure = scores['rouge1'].fmeasure
        model_trained = 'pegasus-wikihow'
        return model_trained, source, predicted, process_title, recall, precision, fmeasure

In [ ]:
wikihow_urls = ['https://www.wikihow.com/Set-Up-a-Tent', 'https://www.wikihow.com/Publish-a-Novel',
             'https://www.wikihow.com/Book-a-Flight', 'https://www.wikihow.com/Communicate-Effectively',
             'https://www.wikihow.com/Treat-Sudden-Cardiac-Arrest', 'https://www.wikihow.com/Doodle',
             'https://www.wikihow.com/Use-the-Rule-of-72', 'https://www.wikihow.com/Train-for-a-Triathlon',
             'https://www.wikihow.com/Use-Crop-Rotation-in-Gardening', 'https://www.wikihow.com/Work-Out-at-Home-Using-Hand-Weights']

In [ ]:
import pandas as pd

dataframe = pd.DataFrame({},columns = ['model_trained_on','text_source','predicted_summary','original_summary',
                                       'recall', 'precision', 'fmeasure','human_evaluation'])

for url in wikihow_urls:
  url1 = url
  model_trained, source, wikihow_prediction, page_title, recall, precision, fmeasure  = summarize_wikihow_text('wikihow',url1)
  dataframe = dataframe.append({'model_trained_on': model_trained, 'text_source':source, 
                                'original_summary': page_title, 'predicted_summary': wikihow_prediction,
                                'recall': recall, 'precision': precision, 
                                'fmeasure': fmeasure
                                },ignore_index=True)

In [ ]:
print('Model used for training: ', model_trained)
print('Source for the text: ', source)
print('Prediction: ',wikihow_prediction)
print('Title: ', page_title)
print('Recall: ',recall)
print('Precision: ',precision)
print('fmeasure: ',fmeasure)

Model used for training:  pegasus-wikihow
Source for the text:  wikihow
Prediction:  Go to the wikiHow article How to Workout with Hand Weights by Personal Trainer co-author Monica Morris for more information on how to workout with hand weights.
Title:  How to Work Out at Home Using Hand Weights
Recall:  0.4444444444444444
Precision:  0.14285714285714285
fmeasure:  0.21621621621621623


In [ ]:
news_urls = ['https://www.abc.net.au/news/2021-11-01/french-macron-accuses-morrison-of-lying-submarine-contract/100584196',
'https://www.bbc.com/news/world-asia-59029517',
'https://thenewdaily.com.au/news/2021/10/31/australia-indonesia-energy-climate/',
'https://www.9news.com.au/world/south-pacific-island-tonga-records-first-case-of-covid-19-prompting-people-to-get-vaccinated-and-warnings-of-a-national-lockdown/a06d7580-a160-4ad1-a2ae-d287edfba62d',
'https://www.abc.net.au/news/2021-11-01/australia-climate-change-reputation-ahead-of-cop26-warming-talks/100579386',
'https://www.9news.com.au/national/coronavirus-international-travel-update-november-1-australian-borders-open-many-still-cant-get-home-exclusive/6a1ac54e-cfaa-40e2-b1b7-9bca7c14d60f',
'https://www.theguardian.com/australia-news/2021/oct/31/new-zealand-australia-quarantine-free-travel-coronavirus',
'https://7news.com.au/news/crime/knife-acid-attack-on-japan-train-reports-c-4381302',
'https://www.abc.net.au/news/2021-10-31/qld-coronavirus-covid-zero-cases-vaccination-push/100578580',
'https://thenewdaily.com.au/news/politics/2021/11/01/scott-morrison-glasgow-cop26/']

In [ ]:
for url in news_urls:
  #print(url)
  url1 = url
  model_trained, source, wikihow_prediction, page_title, recall, precision, fmeasure  = summarize_wikihow_text('news',url1)
  dataframe = dataframe.append({'model_trained_on': model_trained, 'text_source':source,
                                'original_summary': page_title, 'predicted_summary': wikihow_prediction,
                                'recall': recall, 'precision': precision, 
                                'fmeasure': fmeasure
                                },ignore_index=True)

In [ ]:
# adding human evaluation score for the text and their summary to the dataframe
# human evaluation done by the team member

dataframe['human_evaluation'] = [2,4,5,1,1,2,5,4,4,5,
                                 1,2,1,4,5,1,5,4,3,3]

In [ ]:
dataframe.to_csv('bhag.csv',index=False)

In [ ]:
dataframe

,model_trained_on,text_source,predicted_summary,original_summary,recall,precision,fmeasure,human_evaluation
0,pegasus-wikihow,wikihow,Learn how to set up a tent with this wikiHow a...,How to Set Up a Tent,1.000000,0.193548,0.324324,2
1,pegasus-wikihow,wikihow,Know that this article is co-written by 24 aut...,How to Publish a Novel,0.200000,0.038462,0.064516,4
2,pegasus-wikihow,wikihow,Learn how to read an airline’s website.<n>Lear...,How to Book a Flight,1.000000,0.185185,0.312500,5
3,pegasus-wikihow,wikihow,Follow the steps in this wikiHow article by fo...,How to Communicate Effectively,0.000000,0.000000,0.000000,1
4,pegasus-wikihow,wikihow,Understand how this article was medically revi...,How to Treat Sudden Cardiac Arrest,0.166667,0.032258,0.054054,1
5,pegasus-wikihow,wikihow,Learn about the authors and the wikiHow commun...,How to Doodle,0.333333,0.035714,0.064516,2
6,pegasus-wikihow,wikihow,Use the Rule of 72 to estimate the number of y...,How to Use the Rule of 72,0.857143,0.193548,0.315789,5
7,pegasus-wikihow,wikihow,Follow the example of one of our co- authors t...,How to Train for a Triathlon,0.666667,0.111111,0.190476,4
8,pegasus-wikihow,wikihow,Learn how to use crop rotation to grow differe...,How to Use Crop Rotation in Gardening,0.857143,0.206897,0.333333,4
9,pegasus-wikihow,wikihow,Go to the wikiHow article How to Workout with ...,How to Work Out at Home Using Hand Weights,0.444444,0.142857,0.216216,5
